# HW3

## Text Processing

### Q1

1. Modify the code I wrote in lecture 8 with what you have learnt in lecture 9 and correctly tokenize the text both on the word and sentence level, and by removing the stopwords. Rewrite the `getSummary` function and all the other functions that it depends by maing these corrections.

2. Rewrite the code I wrote for `getKeywords` function making the same corrections.

3. Test your code from parts 1 and 2 on random articles from the Guardian.

4. Rewrite the `getSubjectGuardian` function for another newspaper in English, and test your code from part 1 and 2 on random articles from this new newspaper.

In [1]:
import requests
import nltk
import regex as re
import spacy
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from bs4 import BeautifulSoup
from xmltodict import parse
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA

# Getting the data
Take data by subject.


In [2]:
def getSubjectGuardian(subject):
    with requests.get(f'https://www.theguardian.com/{subject}/rss') as link:
        raw = parse(link.text)
    return raw['rss']['channel']['item']

We need to extract text from HTML and combine individual paragraphs. 

In [3]:
def getText(url):
    with requests.get(url) as link:
        raw = BeautifulSoup(link.content,'html.parser')
    return ' '.join([x.text for x in raw.find_all('p')])

In [4]:
tech=getSubjectGuardian('technology')

This is our text.

In [5]:
text=getText(tech[0]['link'])
text

'World’s richest person aims to reassure followers, saying he will not seek to politicise social media network Elon Musk said Twitter must be “politically neutral” in a comment posted last night after a wave of account deletions by left-leaning users on the social media network. In the days since Musk’s $44bn (£34.5bn) acquisition offer was accepted by Twitter’s board, hundreds of thousands of users have closed their accounts on the site, the company confirmed, leading to a dip in follower numbers for left-leaning politicians and celebrities such as Barack and Michelle Obama, Taylor Swift and Jeremy Corbyn. Meanwhile, rightwing influencers such as the far-right congresswoman Marjorie Taylor Greene, Boris Johnson and Ted Cruz have all had large gains as new users sign up for the service. In the latest of a series of tweets about the platform Musk hopes to take into private ownership, the world’s wealthiest person suggested he would not seek to politicise Twitter. “For Twitter to deserve

# tokenize the text
tokenize the text by word.

In [6]:
words= word_tokenize(text)

Stop Words in English.

In [7]:
swEN = set(stopwords.words('english'))

Removing the stopwords from words.

In [8]:
reduced = [word for word in words if word not in swEN]
reduced

['World',
 '’',
 'richest',
 'person',
 'aims',
 'reassure',
 'followers',
 ',',
 'saying',
 'seek',
 'politicise',
 'social',
 'media',
 'network',
 'Elon',
 'Musk',
 'said',
 'Twitter',
 'must',
 '“',
 'politically',
 'neutral',
 '”',
 'comment',
 'posted',
 'last',
 'night',
 'wave',
 'account',
 'deletions',
 'left-leaning',
 'users',
 'social',
 'media',
 'network',
 '.',
 'In',
 'days',
 'since',
 'Musk',
 '’',
 '$',
 '44bn',
 '(',
 '£34.5bn',
 ')',
 'acquisition',
 'offer',
 'accepted',
 'Twitter',
 '’',
 'board',
 ',',
 'hundreds',
 'thousands',
 'users',
 'closed',
 'accounts',
 'site',
 ',',
 'company',
 'confirmed',
 ',',
 'leading',
 'dip',
 'follower',
 'numbers',
 'left-leaning',
 'politicians',
 'celebrities',
 'Barack',
 'Michelle',
 'Obama',
 ',',
 'Taylor',
 'Swift',
 'Jeremy',
 'Corbyn',
 '.',
 'Meanwhile',
 ',',
 'rightwing',
 'influencers',
 'far-right',
 'congresswoman',
 'Marjorie',
 'Taylor',
 'Greene',
 ',',
 'Boris',
 'Johnson',
 'Ted',
 'Cruz',
 'large',
 'ga

In [9]:
def processText(text):
    sentences = re.split(r'[.?!]',text)
    return [re.sub(r'[^\w\s]','',x.lower()) for x in sentences]

In [10]:
def getMatrix(sentences):
    vectorizer = CountVectorizer()
    return vectorizer.fit_transform(sentences)

In [11]:
def getSummary(text,k):
    sentences = processText(text)
    matrix = getMatrix(sentences)
    projection = PCA(n_components=1)
    weights = projection.fit_transform(matrix.toarray())
    res = list(zip(weights.transpose()[0],range(112),sentences))
    tmp = sorted(res,key=lambda x: x[0],reverse=True)[:k]
    return sorted(tmp, key=lambda x: x[1])

In [12]:
getSummary(text,6)

[(4.516465690943847,
  10,
  ' the former president donald trump was banned from the social network in 2021 for his role in encouraging the storming of the us capitol while greene had her personal account permanently suspended after breaking the platforms fivestrike rule'),
 (4.474975719233619,
  16,
  ' katy perry the pop star who is the sites third biggest user lost 7000 followers in a few days while obama whose 132 million followers make the former us presidents account the most popular on the site lost 5000 on tuesday alone'),
 (1.9216698829643972,
  18,
  ' cruz the junior senator for texas added more than 60000 followers while greene gained more than 100000 followers in the last week a tenfold increase in the normal rate'),
 (3.893372354309854,
  20,
  ' some speculated that the changes were the result of action from twitter cleaning house in preparation for the acquisition but the social network said they were the result of organic activity'),
 (2.5005054706430534,
  29,
  ' the

### Q2

Write a function that returns all named entities (proper names, country names, corporation names only) from a URL. Function should take the URL as the input and must return the list of named entities from that URL. Test your code on random articles from the Guardian. Don't use the NLTK's NER that I demonstrated during the lecture. Use the SpaCY's NER function.

extract named entitites from a text by using spacy library.

In [13]:
NER = spacy.load("en_core_web_sm")

we get text data from url.Create 3 empty arrays to hold the entity names. thanks to spacy library,we have labels. Using this labels,we compare the tags with the PERSON, ORGANIZATION and GEOPOLITICAL ENTITY.  

In [14]:
def function(url):
    with requests.get(url) as link:
        raw = BeautifulSoup(link.content,'html.parser')
    txt=' '.join([x.text for x in raw.find_all('p')])
    res = NER(txt)
    person=[]
    country=[]
    corp=[]
    for word in res.ents:
        if word.label_== 'PERSON':
            person.append(word.text)
        if word.label_== 'GPE':
            country.append(word.text)
        if word.label_== 'ORG':
            corp.append(word.text)
    return person,country,corp

In [15]:
person,country,corp=function(tech[0]['link'])

In [16]:
print('PERSON: \n' ,person)
print('COUNTRY:\n',country)
print('CORPORATION: \n',corp)

PERSON: 
 ['Musk', 'Barack', 'Michelle Obama', 'Taylor Swift', 'Jeremy Corbyn', 'Marjorie Taylor Greene', 'Boris Johnson', 'Ted Cruz', 'Musk', 'Musk', 'Musk', 'Donald Trump', 'Musk', 'Ben Shapiro', 'Katy Perry', 'Michelle Obama', 'Cruz', 'Johnson', 'Jeremy Corbyn', 'Musk', 'Dick Costolo', 'Musk', 'Musk', 'Musk', 'Musk']
COUNTRY:
 ['US', 'US', '~80', 'US', 'Texas']
CORPORATION: 
 ['Twitter', 'Twitter', 'Twitter', 'Greene', 'Greene', 'Twitter', 'Labour', 'Twitter', 'Vijaya Gadde', 'Twitter', 'Twitter', 'the Federal Trade Commission', 'Reuters', 'Information', 'Twitter']


### Q3

1. Write a function that returns the most positive and the most negative sentences from a text. The function must take the text as the input and must return a 2-tuple: the first element as the most positive and the second as the most negative sentence with their polarity scores.

2. Test your function on random articles from the Guardian.

Assigning the SentimentIntensityAnalyzer() as a analyzer variable.

In [17]:
analyzer = SentimentIntensityAnalyzer()

we analyze tokenized text using.Then we find the indexes of the most positive and negative sentences. 

In [18]:
def finds(text):
    sentences = sent_tokenize(text)
    a=[]
    for x in sentences:
        a.append(analyzer.polarity_scores(x))
    pozitif=[]
    negatif=[]
    for i in a:
        pozitif.append(i['pos'])
        negatif.append(i['neg'])
    indexp=pozitif.index(max(pozitif))
    indexn=negatif.index(max(negatif))
    for i in sentences:
        if analyzer.polarity_scores(i)==a[indexp]:
            poz=i
        if analyzer.polarity_scores(i)==a[indexn]:
            neg=i
    return poz ,neg

we use the technology text in the first question

In [19]:
poz ,neg=finds(text)
print('The most positive sentence from a text: \n',poz)
print('The most negative sentence from a text: \n',neg)

The most positive sentence from a text: 
 In earlier comments, Musk has been outspoken about his desire to promote free speech on Twitter, saying that he is “against censorship that goes far beyond the law”.
The most negative sentence from a text: 
 His tweets sparked tens of thousands of abusive messages targeted at the executive, and a public rebuke from a former Twitter chief executive, Dick Costolo.
